In [1]:
import torch
print(torch.cuda.is_available()) 

In [2]:
# coding:utf-8
# 导入必备的工具包
from langchain.prompts import PromptTemplate
from get_vector import *
from model import ChatGLM2

# 加载FAISS向量库
EMBEDDING_MODEL = './m3e-base'
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
db = FAISS.load_local(r'faiss/apiDocs', embeddings, allow_dangerous_deserialization=True)


def get_related_content(related_docs):
    related_content = []
    for doc in related_docs:
        related_content.append(doc.page_content.replace('\n\n', '\n'))
    return '\n'.join(related_content)


def define_prompt():
    question = '调用api，创建一个x=1,y=1,z=1的节点'
    docs = db.similarity_search(question, k=1)
    # print(f'docs-->{docs}')
    related_docs = get_related_content(docs)

    # 构建模板
    PROMPT_TEMPLATE = """
           基于以下已知信息，简洁和专业的来回答用户的问题。不允许在答案中添加编造成分。
           已知内容:
           {context}
           问题:
           {question}"""
    prompt = PromptTemplate(input_variables=["context", "question"],
                            template=PROMPT_TEMPLATE)

    my_prompt = prompt.format(context=related_docs, question=question)
    return my_prompt

In [4]:
llm = ChatGLM2()
llm.load_model(r'C:\Users\65197\Downloads\chatglm')

In [5]:
my_prompt = define_prompt()
result = llm(my_prompt)
print(f'result-->{result}')

In [7]:
my_prompt = '1+1等于几'
result = llm(my_prompt)
print(f'result-->{result}')